Considering Adding:
- Confidence intervals for performance metrics (Most papers only really seem to go this far)
- Statistical significance tests between different approaches
- Variance analysis across multiple runs




In [3]:
!pip install optuna

In [4]:
import sys
print(sys.executable)


import optuna
print(optuna.__version__)

/home/sd37/.conda/envs/thesis/bin/python
4.1.0


/home/sd37/.conda/envs/thesis/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### **This experiment is investigating the performance of an adaptive reward function to state of the art reward functions in environments with environmentally variable changes.**

-> Is there a statisitcally significant improvement in performance over time in this varying environment.


In [5]:
import gymnasium as gym
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime
import os
from pathlib import Path

import sys
from pathlib import Path


current_dir = os.getcwd()  
project_root = str(Path(current_dir).parent.parent)
sys.path.append(project_root)


# Initialize environment and device
from AdaptiveRewardFunctionLearning.Prompts.prompts import device, apiKey,modelName

#Cu stomCartPoleEnv
from RLEnvironment.env import CustomCartPoleEnv
#RewardUpdateSystem
from AdaptiveRewardFunctionLearning.RewardGeneration.rewardCritic import RewardUpdateSystem
#DQLearningAgent
from RLEnvironment.training.agent import DQLearningAgent
from RLEnvironment.training.training import trainDQLearning

#DynamicRewardFunction
from AdaptiveRewardFunctionLearning.RewardGeneration.rewardCodeGeneration import dynamicRewardFunction

#import
from AdaptiveRewardFunctionLearning.Visualisation.trainingTestFunctions import (
    runEpisode,
    detectJumps,
    analyzeRewardSensibility,
    performUpdate,
    updateCompositeRewardFunction,
    plotExperimentResults,
    savePlot
)

# Import new reward functions
from AdaptiveRewardFunctionLearning.RewardGeneration.cartpole_energy_reward import EnergyBasedRewardFunction
from AdaptiveRewardFunctionLearning.RewardGeneration.cartpole_meta_learning import meta_learning_cartpole
from AdaptiveRewardFunctionLearning.RewardGeneration.reward_meta_learning import RewardFunctionMetaLearner

**State of the Art Reward Functions with Reference to Papers**

**Potential-based Reward Shaping (PBRS):**
```python
def potentialBasedRewardShaping(observation, action):
    x, xDot, angle, angleDot = observation
    gamma = 0.99  # Example discount factor

    def phi(x, xDot, angle, angleDot):
        # Example potential function
        return -abs(x) - abs(angle)

    current_potential = phi(x, xDot, angle, angleDot)
    next_potential = phi(x + xDot, angle + angleDot, xDot, angleDot)  # Simplified next state
    return float(gamma * next_potential - current_potential)
```

Paper: "Potential-based Shaping in Model-based Reinforcement Learning"

Link: https://cdn.aaai.org/AAAI/2008/AAAI08-096.pdf


**Parameterized Reward Shaping:**
```python
def parameterizedRewardShaping(observation, action):
    x, xDot, angle, angleDot = observation
    original_reward = 1.0  # Assuming default CartPole reward

    def f(x, xDot, angle, angleDot):
        # Example shaping reward function
        return -abs(angle)

    def z_phi(x, xDot, angle, angleDot):
        # Example shaping weight function
        return 0.5

    shaping_reward = f(x, xDot, angle, angleDot)
    shaping_weight = z_phi(x, xDot, angle, angleDot)
    return float(original_reward + shaping_weight * shaping_reward)
```

Paper: "Learning to Utilize Shaping Rewards: A New Approach of Reward Shaping"

Link: http://arxiv.org/pdf/2011.02669.pdf


**Energy Based Reward Function - Physics Based**

```python
def energyBasedReward(observation, action):
    x, xDot, angle, angleDot = observation
    
    # Calculate kinetic and potential energy components
    kineticEnergy = 0.5 * (xDot**2 + angleDot**2)
    potentialEnergy = 9.8 * (1 + cos(angle))  # g * h, where h depends on angle
    
    # Reward is inverse of total energy (less energy = more stable = better reward)
    energyPenalty = -(kineticEnergy + potentialEnergy)
    return float(1.0 + 0.1 * energyPenalty)  # Base reward plus energy term
```

Paper: "Energy-Based Control for Safe Robot Learning" (2019)

Link: https://ieeexplore.ieee.org/document/8794207


**Baseline Reward Function:**
```python
def baselineCartPoleReward(observation, action):
    return 1.0
```

### **Performance Experiment**

In [6]:

# Initialize reward functions and meta-learners
energy_reward = EnergyBasedRewardFunction(mass_cart=1.0, mass_pole=0.1, length=0.5, gravity=9.8)
meta_reward = RewardFunctionMetaLearner(state_dim=4, action_dim=1)  # CartPole has 4 state dims, 1 action dim

def potentialBasedRewardShaping(observation, action):
    """Advanced potential-based reward shaping using meta-learning"""
    reward_func = meta_reward.generate_reward_function()
    return float(reward_func(observation, action))

def parameterizedRewardShaping(observation, action):
    """Meta-learning based parameterized reward shaping"""
    # Use meta-learning framework for parameter optimization
    learner = meta_learning_cartpole()
    return float(learner.parameterized_reward(observation, action))

def energyBasedReward(observation, action):
    """Enhanced physics-based energy reward"""
    return float(energy_reward.compute_reward(observation, action))

def baselineReward(observation, action):
    """Standard baseline reward"""
    return 1.0

/home/sd37/.conda/envs/thesis/lib/python3.10/site-packages/torch/optim/lr_scheduler.py:62: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(


In [7]:
def runPerformanceComparisonTest(
    episodes=1000, 
    changeInterval=500, 
    lengthchanges=[0.5, 1.5],
    mass_cart=1.0,
    mass_pole=0.1,
    initial_length=0.5,
    gravity=9.8
):
    print("Starting Performance Comparison Test...")
    
    currentlengthidx = 0
    
    env = gym.make('CartPole-v1', max_episode_steps=2000)
    env = CustomCartPoleEnv(env)
    env.setEnvironmentParameters(masscart=mass_cart, length=lengthchanges[0], gravity=gravity)
    
    energy_reward = EnergyBasedRewardFunction(
        mass_cart=mass_cart, 
        mass_pole=mass_pole, 
        length=initial_length, 
        gravity=gravity
    )
    meta_reward = RewardFunctionMetaLearner(state_dim=4, action_dim=1)
    
    rewardfunctions = {
        'adaptivereward': {
            'agent': DQLearningAgent(env, 4, 2, device),
            'updatesystem': RewardUpdateSystem(apiKey, modelName), 
            'rewardfunction': None,
            'update_method': 'llm'
        },
        'meta_learning': {
            'agent': DQLearningAgent(env, 4, 2, device),
            'updatesystem': meta_reward,
            'rewardfunction': meta_reward.generate_reward_function(),
            'update_method': 'meta'
        },
        'energy_based': {
            'agent': DQLearningAgent(env, 4, 2, device),
            'updatesystem': energy_reward,
            'rewardfunction': energy_reward.compute_reward,
            'update_method': 'physics'
        },
        'baseline': {                           # Add this new entry
            'agent': DQLearningAgent(env, 4, 2, device),
            'updatesystem': None,               # No update system needed
            'rewardfunction': baselineReward,   # Use our simple function
            'update_method': None               # No updates needed
        }
    }

    
    results = {}
    
    for rewardname, rewardinfo in rewardfunctions.items():
        print(f"\nTesting reward function: {rewardname}")
        
        env.reset()
        if rewardname != 'adaptivereward':
            env.setRewardFunction(rewardinfo['rewardfunction'])
        else:
            # Reset lastUpdateEpisode for adaptive reward at start of its run
            rewardinfo['updatesystem'].lastUpdateEpisode = 0
        
        episoderewards = []
        episodebalancetimes = []
        rewardchangeepisodes = []
        
        def onEpisodeEnd(env, updatesystem, episode, reward, steps):
            nonlocal episoderewards, episodebalancetimes, rewardchangeepisodes, currentlengthidx
            
            # Record episode results
            episoderewards.append(reward)
            episodebalancetimes.append(steps)
            
            # Create metrics dictionary
            metrics = {
                'currentEpisode': episode,
                'recentRewards': episoderewards[-100:] if len(episoderewards) > 100 else episoderewards,
                'averageBalanceTime': np.mean(episodebalancetimes[-100:]) if episodebalancetimes else 0,
                'balanceTimeVariance': np.var(episodebalancetimes[-100:]) if len(episodebalancetimes) > 1 else 0
            }
            
            # Debug print for metrics
            if episode % 1000 == 0:
                print(f"\nMetrics Debug at Episode {episode}:")
                print(f"Recent Average Reward: {np.mean(metrics['recentRewards']):.2f}")
                print(f"Average Balance Time: {metrics['averageBalanceTime']:.2f}")
                print(f"Balance Time Variance: {metrics['balanceTimeVariance']:.2f}")
            
            # Handle different update methods
            if rewardinfo['update_method'] == 'llm':
                # Debug print for LLM update check
                if episode % 1000 == 0:
                    print(f"Last Update Episode: {updatesystem.lastUpdateEpisode}")
                    print(f"Time Since Last Update: {episode - updatesystem.lastUpdateEpisode}")
                
                if updatesystem.waitingTime('stability', metrics, updatesystem.lastUpdateEpisode):
                    print(f"\nAttempting LLM update at episode {episode}")
                    updateCompositeRewardFunction(env, updatesystem, metrics, dynamicRewardFunction)
                    
                    if hasattr(env.rewardFunction, 'compositeHistory'):
                        latest_updates = [update['episode'] for update in env.rewardFunction.compositeHistory 
                                        if update['episode'] == episode]
                        if latest_updates:
                            rewardchangeepisodes.append(episode)
                            updatesystem.lastUpdateEpisode = episode
                            print(f"✓ LLM update successful at episode {episode}")
                            print(f"Current reward changes: {rewardchangeepisodes}")
            
            # In onEpisodeEnd function:
            if rewardinfo['update_method'] == 'meta':
                # Update every 1000 episodes instead of at environment change
                if episode % 1000 == 0 and episode > 0:
                    print(f"\nAttempting meta-learning update at episode {episode}")
                    recent_memories = list(agent.memory)[-100:] if len(agent.memory) >= 100 else list(agent.memory)
                    trajectory = {
                        'states': recent_memories,
                        'episode_length': steps,
                        'total_reward': reward,
                        'env_params': {
                            'length': env.env.length,
                            'mass_cart': env.env.masscart
                        }
                    }
                    meta_loss = updatesystem.meta_update([trajectory])
                    env.setRewardFunction(updatesystem.generate_reward_function())
                    rewardchangeepisodes.append(episode)
                    print(f"✓ Meta-learning update completed, loss: {meta_loss}")
            
            elif rewardinfo['update_method'] == 'physics':
                if episode % changeInterval == 0 and episode > 0:
                    print(f"\nUpdating physics-based reward at episode {episode}")
                    updatesystem.length = lengthchanges[currentlengthidx]
                    env.setRewardFunction(updatesystem.compute_reward)
                    rewardchangeepisodes.append(episode)
                    print("✓ Physics-based update completed")
            
            # Environment changes
            if episode % changeInterval == 0 and episode > 0:
                currentlengthidx = (currentlengthidx + 1) % len(lengthchanges)
                newlength = lengthchanges[currentlengthidx]
                env.setEnvironmentParameters(length=newlength)
                print(f"\nChanged pole length to: {newlength}m at episode {episode}")
            
            # Regular progress updates
            if episode % 10000 == 0:
                print(f"\nEpisode {episode}")
                print(f"  Steps: {steps}")
                print(f"  Total Reward: {reward}")
                print(f"  Average Balance Time = {np.mean(episodebalancetimes[-50:]):.2f}")
                print(f"  Average Reward = {np.mean(episoderewards[-50:]):.2f}")
        
        agent, env, rewards = trainDQLearning(
            agent=rewardinfo['agent'],
            env=env,
            numEpisodes=episodes,
            updateSystem=rewardinfo['updatesystem'],
            onEpisodeEnd=onEpisodeEnd
        )
        
        results[rewardname] = {
            'rewards': episoderewards,
            'balancetimes': episodebalancetimes,
            'rewardChanges': rewardchangeepisodes
        }
        
        print(f"\nCompleted testing {rewardname}")
        print(f"Final average reward: {np.mean(episoderewards[-100:]):.2f}")
        print(f"Final average balance time: {np.mean(episodebalancetimes[-100:]):.2f}")
    
    return results

In [8]:
# Run Experiment

changeInterval = 20000

# results = runPerformanceComparisonTest(
#     episodes=40000,  
#     changeInterval=changeInterval,
#     mass_cart=1.0,
#     lengthchanges=[0.3, 0.9]  
# )

In [9]:
def visualizePerformanceComparison(results, changeInterval, folder_path):
    """
    Create and save performance comparison visualizations
    """
    fig, (ax1, ax2, ax3) = plt.subplots(3, 1, figsize=(12, 15))
    
    # Color map for different reward functions
    colors = ['b', 'g', 'r', 'c', 'm']
    
    # Plot rewards for each reward function with variance
    for idx, (rewardname, rewardresults) in enumerate(results.items()):
        rewards = pd.Series(rewardresults['rewards'])
        
        # Calculate rolling mean and standard deviation for rewards
        window = 50
        rolling_mean_rewards = rewards.rolling(window=window).mean()
        rolling_std_rewards = rewards.rolling(window=window).std()
        
        # Plot mean line for rewards
        ax1.plot(rolling_mean_rewards, 
                label=f'{rewardname}', 
                linewidth=2, 
                color=colors[idx])
        
        # Plot variance area for rewards
        ax1.fill_between(
            range(len(rewards)),
            rolling_mean_rewards - rolling_std_rewards,
            rolling_mean_rewards + rolling_std_rewards,
            color=colors[idx],
            alpha=0.2
        )
        
        # Add vertical lines for environment changes (red)
        change_episodes = range(changeInterval, len(rewards), changeInterval)
        for ep in change_episodes:
            ax1.axvline(x=ep, color='r', linestyle='--', alpha=0.3,
                       label='Environment Change' if ep == change_episodes[0] else None)
        
        # Add vertical lines for reward function changes (green)
        if 'rewardChanges' in rewardresults:
            for ep in rewardresults['rewardChanges']:
                ax1.axvline(x=ep, color='g', linestyle='--', alpha=0.3,
                          label=f'{rewardname} Update' if ep == rewardresults['rewardChanges'][0] else None)
    
    ax1.set_title('Average Reward Over Time with Variance')
    ax1.set_xlabel('Episode')
    ax1.set_ylabel('Reward')
    ax1.legend()
    ax1.grid(True)
    
    # Plot balance times
    for idx, (rewardname, rewardresults) in enumerate(results.items()):
        balancetimes = pd.Series(rewardresults['balancetimes'])
        
        rolling_mean_balance = balancetimes.rolling(window=window).mean()
        rolling_std_balance = balancetimes.rolling(window=window).std()
        
        ax2.plot(rolling_mean_balance,
                label=f'{rewardname}', 
                linewidth=2, 
                color=colors[idx])
        
        ax2.fill_between(
            range(len(balancetimes)),
            rolling_mean_balance - rolling_std_balance,
            rolling_mean_balance + rolling_std_balance,
            color=colors[idx],
            alpha=0.2
        )
        
        # Add vertical lines for environment changes
        for ep in change_episodes:
            ax2.axvline(x=ep, color='r', linestyle='--', alpha=0.3,
                       label='Environment Change' if ep == change_episodes[0] else None)
        
        # Add vertical lines for reward function changes
        if 'rewardChanges' in rewardresults:
            for ep in rewardresults['rewardChanges']:
                ax2.axvline(x=ep, color='g', linestyle='--', alpha=0.3,
                          label=f'{rewardname} Update' if ep == rewardresults['rewardChanges'][0] else None)
    
    ax2.set_title('Average Balance Time Over Episodes with Variance')
    ax2.set_xlabel('Episode')
    ax2.set_ylabel('Steps')
    ax2.legend()
    ax2.grid(True)
    
    # Plot environment parameters
    env_param_history = []
    for episode in range(len(next(iter(results.values()))['rewards'])):
        idx = (episode // changeInterval) % 2
        length = 0.3 if idx == 0 else 0.9  # Alternating between 0.3 and 0.9
        env_param_history.append(length)
    
    ax3.plot(env_param_history, label='Pole Length', color='purple')
    ax3.set_title('Environment Parameters Over Episodes')
    ax3.set_xlabel('Episode')
    ax3.set_ylabel('Pole Length (m)')
    ax3.grid(True)
    ax3.legend()
    
    plt.tight_layout()
    
    # Save plots
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    filepath = os.path.join(folder_path, f"performance_comparison_{timestamp}.png")
    plt.savefig(filepath, bbox_inches='tight', dpi=300)
    print(f"Saved plot: performance_comparison_{timestamp}.png in {folder_path}")
    plt.close()
    
    
    
def calculateStability(rewards):
    """
    Calculate stability score based on reward variance in the last 100 episodes
    Lower variance = higher stability
    """
    if len(rewards) < 100:
        return 0.0
    
    last_hundred = rewards[-100:]
    mean_reward = np.mean(last_hundred)
    if mean_reward == 0:
        return 0.0
        
    # Calculate coefficient of variation (normalized standard deviation)
    stability = 1 - (np.std(last_hundred) / mean_reward)
    return max(0, min(1, stability))  # Normalize between 0 and 1

def calculateConvergenceTime(rewards, threshold=195, window=50):
    """
    Calculate the number of episodes needed to reach and maintain a certain performance
    threshold for a given window of episodes
    """
    if len(rewards) < window:
        return len(rewards)
    
    rolling_mean = pd.Series(rewards).rolling(window).mean()
    
    for episode in range(window, len(rewards)):
        if rolling_mean[episode] >= threshold:
            # Check if performance is maintained
            maintained = all(avg >= threshold * 0.9 for avg in rolling_mean[episode:episode+window])
            if maintained:
                return episode
    
    return len(rewards)  # If never converged, return total episodes

def calculatePerformanceMetrics(results):
    metrics = {}
    for rewardname, rewardresults in results.items():
        metrics[rewardname] = {
            'finalavgreward': np.mean(rewardresults['rewards'][-100:]),
            'finalavgbalance': np.mean(rewardresults['balancetimes'][-100:]),
            'convergencetime': calculateConvergenceTime(rewardresults['rewards']),
            'stability': calculateStability(rewardresults['rewards'])
        }
    return pd.DataFrame(metrics).T
    
    
    
    
    
def saveMetricsTable(metrics, filename, folder_path):
    """Save metrics table to specified folder"""
    # Create figure for metrics table
    fig, ax = plt.subplots(figsize=(12, 4))
    ax.axis('tight')
    ax.axis('off')
    
    # Create table with formatted metrics
    table = ax.table(
        cellText=metrics.values.round(3),
        colLabels=metrics.columns,
        rowLabels=metrics.index,
        cellLoc='center',
        loc='center'
    )
    
    # Adjust font size and scaling
    table.auto_set_font_size(False)
    table.set_fontsize(9)
    table.scale(1.2, 1.5)
    
    plt.title("Performance Metrics Comparison")
    
    # Save with timestamp
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    filepath = os.path.join(folder_path, f"{filename}_{timestamp}.png")
    plt.savefig(filepath, bbox_inches='tight', dpi=300)
    print(f"Saved metrics table: {filename}_{timestamp}.png in {folder_path}")
    plt.close()



In [10]:

# Visualize the results
# visualizePerformanceComparison(results,changeInterval)


# # Calculate and display the metrics
# metrics = calculatePerformanceMetrics(results)
# print("\nPerformance Metrics:")
# print(metrics)
    
# Call the function
# saveMetricsTable(metrics)

### Multiple runs to generate confidence intervals

In [11]:
def createExperimentFolder():
    """Create a timestamped folder for experiment results"""
    from datetime import datetime
    import os
    
    # Create base experiment folder if it doesn't exist
    if not os.path.exists("PerformanceExperiment"):
        os.makedirs("PerformanceExperiment")
    
    # Create timestamped subfolder
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    experiment_folder = os.path.join("PerformanceExperiment", f"experiment_{timestamp}")
    os.makedirs(experiment_folder)
    
    return experiment_folder

def savePlot(fig, filename, folder_path):
    """Save plot to specified folder with timestamp"""
    from datetime import datetime
    import os
    
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    filepath = os.path.join(folder_path, f"{filename}_{timestamp}.png")
    fig.savefig(filepath, bbox_inches='tight', dpi=300)
    print(f"Saved plot: {filename}_{timestamp}.png in {folder_path}")
    
def saveMetricsTable(metrics, filename, folder_path):
    """Save metrics table to specified folder"""
    import matplotlib.pyplot as plt
    from datetime import datetime
    import os
    
    # Style the DataFrame for visualization
    fig, ax = plt.subplots(figsize=(12, 4))
    ax.axis('tight')
    ax.axis('off')
    table = ax.table(cellText=metrics.values,
                    colLabels=metrics.columns,
                    rowLabels=metrics.index,
                    cellLoc='center',
                    loc='center')
    
    # Adjust font size and scaling
    table.auto_set_font_size(False)
    table.set_fontsize(9)
    table.scale(1.2, 1.5)
    
    plt.title("Performance Metrics Comparison")
    
    # Save with timestamp
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    filepath = os.path.join(folder_path, f"{filename}_{timestamp}.png")
    plt.savefig(filepath, bbox_inches='tight', dpi=300)
    print(f"Saved metrics table: {filename}_{timestamp}.png in {folder_path}")
    plt.close()

def runMultipleExperiments(numRuns=4, episodes=40000, changeInterval=20000):
    """
    Run multiple experiments and save results in organized folders
    """
    # Create main experiment folder
    experiment_folder = createExperimentFolder()
    
    allResults = []
    allMetrics = []
    aggregatedMetrics = {
        'adaptivereward': [],
        'meta_learning': [],
        'energy_based': []
    }
    
    # Create run folders
    for run in range(numRuns):
        print(f"\nStarting Run {run + 1}/{numRuns}")
        
        # Create folder for this run
        run_folder = os.path.join(experiment_folder, f"run_{run + 1}")
        os.makedirs(run_folder)
        
        # Run experiment
        results = runPerformanceComparisonTest(
            episodes=episodes,
            changeInterval=changeInterval,
            mass_cart=1.0,
            lengthchanges=[0.3, 0.9]
        )
        
        # Calculate metrics
        metrics = calculatePerformanceMetrics(results)
        
        # Store results
        allResults.append(results)
        allMetrics.append(metrics)
        
        # Store metrics by reward type
        for rewardType in results.keys():
            if rewardType not in aggregatedMetrics:
                aggregatedMetrics[rewardType] = []
            aggregatedMetrics[rewardType].append(metrics.loc[rewardType])
        
        # Visualize and save individual run results
        visualizePerformanceComparison(results, changeInterval, run_folder)
        saveMetricsTable(metrics, f"metrics_run_{run + 1}", run_folder)
    
    # Calculate confidence intervals (95%)
    confidenceIntervals = {}
    
    for rewardType, metrics_list in aggregatedMetrics.items():
        metrics_df = pd.DataFrame(metrics_list)
        confidenceIntervals[rewardType] = {}
        
        for column in metrics_df.columns:
            mean = metrics_df[column].mean()
            ci = 1.96 * metrics_df[column].std() / np.sqrt(numRuns)
            
            confidenceIntervals[rewardType][column] = {
                'mean': mean,
                'ci': ci
            }
    
    # Create final results table
    resultsTable = pd.DataFrame({
        rewardType: {
            metric: f"{data[metric]['mean']:.2f} ± {data[metric]['ci']:.2f}"
            for metric in ['finalavgreward', 'finalavgbalance', 'convergencetime', 'stability']
        }
        for rewardType, data in confidenceIntervals.items()
    }).T
    
    # Save final results in main experiment folder
    saveMetricsTable(resultsTable, "final_results", experiment_folder)
    
    # Save aggregate statistics
    with open(os.path.join(experiment_folder, "aggregate_statistics.txt"), "w") as f:
        f.write("Aggregate Statistics:\n")
        for rewardType, metrics in confidenceIntervals.items():
            f.write(f"\n{rewardType}:\n")
            for metric, values in metrics.items():
                f.write(f"{metric}: {values['mean']:.2f} ± {values['ci']:.2f}\n")
    
    print(f"\nExperiment results saved in: {experiment_folder}")
    return confidenceIntervals, allResults, resultsTable

In [ ]:
confidenceIntervals, allResults, resultsTable = runMultipleExperiments(
    numRuns=4,
    episodes=10000,
    changeInterval=5000
)


Starting Run 1/4
Starting Performance Comparison Test...
Environment parameters updated: masscart=1.0, length=0.3, gravity=9.8

Testing reward function: adaptivereward

Metrics Debug at Episode 0:
Recent Average Reward: 15.92
Average Balance Time: 16.00
Balance Time Variance: 0.00
Last Update Episode: 0
Time Since Last Update: 0

Episode 0
  Steps: 16
  Total Reward: 15.922463417053223
  Average Balance Time = 16.00
  Average Reward = 15.92
